In [ ]:
!pip install syft==0.2.5

     |████████████████████████████████| 369 kB 4.3 MB/s 
     |████████████████████████████████| 484 kB 25.4 MB/s 
     |████████████████████████████████| 753.4 MB 6.6 kB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 200 kB 47.4 MB/s 
     |████████████████████████████████| 4.0 MB 51.9 MB/s 
     |████████████████████████████████| 2.1 MB 33.9 MB/s 
     |████████████████████████████████| 1.8 MB 41.5 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Created wheel for tornado: filename=tornado-4.5.3-cp37-cp37m-linux_x86_64.whl size=434057 sha256=b1db7cb9eab6e738362b6f972da6f5a085aa27e278aeeee752ffa25ac8f68ddc
  Stored in directory: /root/.cache/

In [ ]:
import torch
import syft as sy

# allow pysyft to work its magic on torch tensors
hook = sy.TorchHook(torch)

# create a virtual worker. in an actual setting this would be on a different machine
client = sy.VirtualWorker( hook, id='client' )

# define a tensor and send it to the client
x = torch.tensor([1,2,3,4,5])
# this leaves us with a pointer to the tensor
x_pointer = x.send( client )

# check out some meta data
print( x_pointer )
print( client._objects )

# we can use this pointers like normal tensors
result = x_pointer + x_pointer
print( result )

# if we want the result we can call get() to send the tensor back to us
result_local = result.get()
# once we call get() it removes the tensor from the other side and our pointer
# becomes invalid
print( result_local )
print( client._objects )
# print( result )

(Wrapper)>[PointerTensor | me:66264475649 -> client:2558623762]
{67560727805: <Plan Plan id:67560727805 owner:client Tags: #fss_eq_plan_1 built>
, 99197118697: <Plan Plan id:99197118697 owner:client Tags: #fss_eq_plan_2 built>
, 31906307257: <Plan Plan id:31906307257 owner:client Tags: #fss_comp_plan_1 built>
, 69402026607: <Plan Plan id:69402026607 owner:client Tags: #fss_comp_plan_2 built>
, 71980366719: <Plan Plan id:71980366719 owner:client Tags: #xor_add_1 built>
, 34977975923: <Plan Plan id:34977975923 owner:client Tags: #xor_add_2 built>
, 2558623762: tensor([1, 2, 3, 4, 5])}
(Wrapper)>[PointerTensor | me:65133425596 -> client:83943614914]
tensor([ 2,  4,  6,  8, 10])
{67560727805: <Plan Plan id:67560727805 owner:client Tags: #fss_eq_plan_1 built>
, 99197118697: <Plan Plan id:99197118697 owner:client Tags: #fss_eq_plan_2 built>
, 31906307257: <Plan Plan id:31906307257 owner:client Tags: #fss_comp_plan_1 built>
, 69402026607: <Plan Plan id:69402026607 owner:client Tags: #fss_comp

In [ ]:
import torch
from torchvision import datasets, transforms  # it may raise errors, and you need restart the runtime
from torch import nn, optim
import syft as sy
hook = sy.TorchHook(torch)

epochs = 15

# Data preprocessing
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.CIFAR100('cifar100', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

torch.manual_seed(0)

# Define our model segments

input_size = 3072
hidden_sizes = [128, 640]
output_size = 100

models = [
    nn.Sequential(
                nn.Linear(input_size, hidden_sizes[0]),
                nn.ReLU(),
                nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                nn.ReLU(),
    ),
    nn.Sequential(
                nn.Linear(hidden_sizes[1], output_size),
                nn.LogSoftmax(dim=1)
    )
]

# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.01,)
    for model in models
]

# create some workers
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
workers = alice, bob

# Send Model Segments to starting locations
model_locations = [alice, bob]

for model, location in zip(models, model_locations):
    model.send(location)

def train(x, target, models, optimizers):
    # Training Logic

    #1) erase previous gradients (if they exist)
    for opt in optimizers:
        opt.zero_grad()

    #2) make a prediction
    a = models[0](x)

    #3) break the computation graph link, and send the activation signal to the next model
    remote_a = a.move(models[1].location, requires_grad=True)

    #4) make prediction on next model using received signal
    pred = models[1](remote_a)

    #5) calculate how much we missed
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)

    #6) figure out which weights caused us to miss
    loss.backward()

    # 7) send gradient of the received activation signal to the model behind
    # grad_a = remote_a.grad.copy().move(models[0].location)

    # 8) backpropagate on bottom model given this gradient
    # a.backward(grad_a)

    #9) change the weights
    for opt in optimizers:
        opt.step()

    #10) print our progress
    return loss.detach().get()

for i in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.send(alice)
        images = images.view(images.shape[0], -1)
        labels = labels.send(bob)
        
        loss = train(images, labels, models, optimizers)
        running_loss += loss

    else:
        print("Epoch {} - Training loss: {}".format(i, running_loss/len(trainloader)))

Files already downloaded and verified
Epoch 0 - Training loss: 4.478142738342285
Epoch 1 - Training loss: 4.085339546203613
Epoch 2 - Training loss: 3.8604989051818848
Epoch 3 - Training loss: 3.7098429203033447
Epoch 4 - Training loss: 3.609361171722412
Epoch 5 - Training loss: 3.5366435050964355
Epoch 6 - Training loss: 3.4761979579925537
Epoch 7 - Training loss: 3.4220638275146484
Epoch 8 - Training loss: 3.374359369277954
Epoch 9 - Training loss: 3.329080581665039
Epoch 10 - Training loss: 3.2829349040985107
Epoch 11 - Training loss: 3.240553379058838
Epoch 12 - Training loss: 3.2006115913391113
Epoch 13 - Training loss: 3.1606180667877197
Epoch 14 - Training loss: 3.123532295227051
